In [2]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [3]:
DATA_DIR = "data"
RAW_DATA_DIR = "data/spotify_dataset.csv"

In [6]:
#standart read csv won't work, commas are in names!!! 
n = 4 #number of columns
df = pd.read_csv(RAW_DATA_DIR,
                 usecols=range(n),
                 lineterminator='\n',
                 header=None)

df = df[1:] #take the data less the header row
df.columns = ["user_id", "artistname", "trackname", "playlistname"] #set the header row as the df header

In [7]:
df

,user_id,artistname,trackname,playlistname
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
5,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010
...,...,...,...,...
12901975,2302bf9c64dc63d88a750215ed187f2c,Mötley Crüe,Wild Side,iPhone
12901976,2302bf9c64dc63d88a750215ed187f2c,John Lennon,Woman,iPhone
12901977,2302bf9c64dc63d88a750215ed187f2c,Tom Petty,You Don't Know How It Feels,iPhone
12901978,2302bf9c64dc63d88a750215ed187f2c,Tom Petty,You Wreck Me,iPhone


## EDA

In [8]:
df.groupby("user_id").agg(
    Count=("trackname", "count"),
    
    TrkUniCnt=("trackname", "nunique"),
    TrkRat=("trackname", lambda x: x.nunique()/x.count()),

    #ArtCnt=("artistname", "count"),
    ArtUniCnt=("artistname", "nunique"),
    ArtRat=("artistname", lambda x: x.nunique()/x.count()),

    #PlyCnt=("playlistname", "count"),
    PlyUniCnt=("playlistname", "nunique"),
    PlyRat=("playlistname", lambda x: x.nunique()/x.count()),
    
)
#FEATURE: COMPARE HOW MANY SONGS A PERSON HAS ADDED TO PLAYLISTS vs uniqueness of songs
# unique artists : num of times an artist was added

/Users/darehunt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


,Count,TrkUniCnt,TrkRat,ArtUniCnt,ArtRat,PlyUniCnt,PlyRat
user_id,,,,,,,
00055176fea33f6e027cd3302289378b,120,120,1.000000,45,0.375000,1,0.008333
0007f3dd09c91198371454c608d47f22,97,95,0.979381,45,0.463918,3,0.030928
000b0f32b5739f052b9d40fcc5c41079,270,266,0.985185,38,0.140741,7,0.025926
000c11a16c89aa4b14b328080f5954ee,716,632,0.882682,363,0.506983,20,0.027933
00123e0f544dee3ab006aa7f1e5725a7,18579,13985,0.752732,1231,0.066261,89,0.004790
...,...,...,...,...,...,...,...
ffe32d5412269f3041c58cbf0dde3306,58,58,1.000000,53,0.913793,2,0.034483
ffec270eae226caa14ddaef291d73fff,32,32,1.000000,4,0.125000,3,0.093750
fff60baf392613ed33f745b89a9b38f7,552,545,0.987319,150,0.271739,42,0.076087


In [9]:
df.groupby("playlistname").agg(
    Count=("user_id", "count"),
    
    UsrUniCnt=("user_id", "nunique"),
    UsrRat=("user_id", lambda x: x.nunique()/x.count()),

    ArtUniCnt=("artistname", "nunique"),
    ArtRat=("artistname", lambda x: x.nunique()/x.count()),

    TrkUniCnt=("trackname", "nunique"),
    TrkRat=("trackname", lambda x: x.nunique()/x.count()),
)

# UsrUniCnt > 1 = 21822

/Users/darehunt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in long_scalars
  


,Count,UsrUniCnt,UsrRat,ArtUniCnt,ArtRat,TrkUniCnt,TrkRat
playlistname,,,,,,,
,46,4,0.086957,39,0.847826,46,1.0
waves,37,1,0.027027,1,0.027027,37,1.0
11,115,1,0.008696,91,0.791304,115,1.0
Frida,23,1,0.043478,6,0.260870,23,1.0
New tunes 05/11,24,1,0.041667,20,0.833333,24,1.0
...,...,...,...,...,...,...,...
🚇,86,1,0.011628,9,0.104651,86,1.0
🚘🚜🚀,39,1,0.025641,30,0.769231,39,1.0
🚙,10,1,0.100000,8,0.800000,10,1.0


In [10]:
df.groupby("trackname").agg(
    Count=("user_id", "count"),
    
    UsrUniCnt=("user_id", "nunique"),
    UsrRat=("user_id", lambda x: x.nunique()/x.count()),

    ArtUniCnt=("artistname", "nunique"),
    ArtRat=("artistname", lambda x: x.nunique()/x.count()),

    PlyUniCnt=("playlistname", "nunique"),
    PlyRat=("playlistname", lambda x: x.nunique()/x.count()),
)


/Users/darehunt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in long_scalars
  
/Users/darehunt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in long_scalars
  # This is added back by InteractiveShellApp.init_path()


,Count,UsrUniCnt,UsrRat,ArtUniCnt,ArtRat,PlyUniCnt,PlyRat
trackname,,,,,,,
""" (Acoustical ) by Everclear""",1,1,1.0,1,1.0,1,1.0
"""Cachaito"" López Y ""Guajiro"" Mirabal De Buena Vista Social Club Y Manuel ""Galbán"" Torralba""",2,1,0.5,1,0.5,1,0.5
15 Years of Tummy Touch Records in Dub,1,1,1.0,1,1.0,1,1.0
Alan Jackson,1,1,1.0,1,1.0,1,1.0
Ashley Tisdale,1,1,1.0,1,1.0,1,1.0
...,...,...,...,...,...,...,...
￾倀愀礀瀀栀漀渀攀,2,1,0.5,1,0.5,2,1.0
￾匀栀甀爀椀欀攀渀,1,1,1.0,1,1.0,1,1.0
￾匀琀爀愀渀最攀 䈀攀栀愀瘀椀漀甀爀 昀琀 吀愀猀栀愀 䈀愀砀琀攀爀 ⠀伀爀椀最椀渀愀氀 䴀椀砀⤀,1,1,1.0,1,1.0,1,1.0


In [11]:
df.groupby(["artistname", "trackname"]).agg({'user_id':['count', lambda x: x.nunique(), lambda x: x.nunique()/x.count()]})


KeyboardInterrupt: 

In [ ]:
# data cleaning
df = df[df['artistname'].apply(lambda x: len(re.sub(r'[a-zA-Z0-9&.!\s\',-üéµ\(\)$+"*\'#]', '', str(x)))) == 0]


## Conversion from Tabular to Network

In [1]:
def read_edge(gph, n0, n1, multiply=True): #add switch between plus and multiply
        if gph.has_edge(n0, n1):
            if multiply:
                gph[n0][n1]['weight'] *= 2
            else:
                gph[n0][n1]['weight'] +=1
            return gph[n0][n1]['weight']
        else:
            gph.add_edge(n0, n1, weight=1) #ADD ANY FEATURES FOR THE NODES!!!
            #nx.set_node_attributes(gph,{"attr": "val"})
            return 1

In [ ]:
G = nx.Graph()

for p, a in df.groupby('paper_id')['author_name']: 
    for a1, a2 in itertools.combinations(a, 2):
    # creates an edge
        read_edge(G, a1, a2)

